# Método probabilistco vs ranked
Primeramente se deben importar las librerías y las funciones para separar los cuentos, eliminar caracteres raros, etc.

In [1]:
import re
import os
import zipfile
import numpy as np
import string 

# Función para poder leer y dividir los archivos .txt
def importar(arc):
    #Leer archivos zip
    files=zipfile.ZipFile(arc,'r')
    files.extractall('')
    cuentos_lista=[]
    cuentos_csv=[]
    cuentos=[]
    for a in files.namelist():
        #cuentos.append(re.)
        aa=re.sub('Cuentos/',"",a)
        if '.xlsx'in aa:
            cuentos_csv.append(aa)
        else:
            cuentos_lista.append(aa)
    # Importar todos los cuentos
    for b in range(len(cuentos_lista)):
        temp=files.read('Cuentos/'+cuentos_lista[b])
        temp=temp.decode('latin-1')
        temp=re.sub(r'[^\w\s]',' ',temp) # Remover signos de puntuacion
        temp=temp.lower() #Convertir a minusculas
        cuentos.append(' '.join(temp.split())) # Se puede convertir a cadena
    
    return cuentos_lista,cuentos_lista,cuentos

In [2]:
# Importar los documentos
cuentos_lista,cuentos_csv,cuentos=importar('C:\\Users\\aldoa\\Procesamiento del lenguaje natural\\1 Practica 1.1\\Cuentos.zip') 

## Método TF-IDF

In [3]:
def create_vocabulary(list_of_documents):
    i = 0
    vocabulary = {}
    # Para cada documento en la lista de documentos
    for document in list_of_documents:
        # Separa el documento en palabras
        words = document.split(' ')
        #words=document
        # Para cada palabra en las palabras
        for word in words:
            # Si la palabra no está en el vocabulario
            if word not in vocabulary:
                vocabulary[word] = i
                i += 1
    return vocabulary

def create_vocabulary_tfidf(list_of_documents):
    i = 0
    vocabulary = {} # Diccionario para almacenar las palabras y el número de palabra
    # Para cada documento en la lista de documentos
    for document_no, document in enumerate(list_of_documents):
        # Separa el documento en palabras
        words = set(document.split(' '))
        # Para cada palabra en las palabras
        for word in words:
            # Si la palabra no está en el vocabulario
            # Agregamos el ID y el documento que aparece
            if word not in vocabulary:
                vocabulary[word] = {'id':i, 'docto':[document_no]} # {ID:número de palabra, DOCTO:número de documento}
                i += 1
            # En otro caso solo agregamos el documento en el que aparece
            else:
                vocabulary[word]['docto'].append(document_no)
        
        for word in vocabulary:
            # df (document frequency) que es el número de documentos que contiene cada palabra
            vocabulary[word]['df'] = len(vocabulary[word]['docto'])
    return vocabulary

def vectorize_tfidf(document,vocabulary, n_docs):
    # Separa el documento en palabras
    words = document.split(' ')
    # Crea un vector de ceros
    vector = np.zeros(len(vocabulary))
    ivector = np.zeros(len(vocabulary))
    # Para cada palabra en las palabras
    for word in  words:
        # Si la palabra está en el vocabulario
        if word in vocabulary:
            # Incrementa el valor del vector en la posición de la palabra
            vector[vocabulary[word]['id']] += 1
            # Vector con frecuencia inversa
            ivector[vocabulary[word]['id']] = n_docs/(vocabulary[word]['df'])
    return (np.log(vector+1)) * ivector #tf-idf

# Función para llamar tfidf
def create_matrix_documents_tfidf(list_of_documents, vocabulary):
    # Crea una matriz de ceros
    matrix = np.zeros((len(list_of_documents), len(vocabulary)))
    # Para cada documento en la lista de documentos
    for i in range(len(list_of_documents)):
        # Crea un vector para el documento
        vector = vectorize_tfidf(list_of_documents[i], vocabulary,len(list_of_documents))
        # Agrega el vector a la matriz
        matrix[i] = vector
    return matrix

def create_vectorized_document(document, vocabulary):
    # Separa el documento en palabras
    words = document.split(' ')
    # Crea un vector de ceros
    vector = np.zeros(len(vocabulary))
    # Para cada palabra en las palabras
    for word in words:
        # Si la palabra está en el vocabulario
        if word in vocabulary:
            # Incrementa el valor del vector en la posición de la palabra
            vector[vocabulary[word]] += 1
    return vector

# Calcula la similaridad de cosenos
def calculate_cosine_similarity(vector1, vector2):
    # Multiplica los vectores
    dot_product = np.dot(vector1, vector2)
    # Calcula la norma de los vectores
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    # Calcula la similitud coseno
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity


def calculate_similarity_for_query(query, matrix_documents, vocabulary):
    # Crea un vector para la consulta
    vector_query = create_vectorized_document(query, vocabulary)
    # Crea una lista para las similitudes
    similarities = {}
    # Para cada documento en la matriz de documentos
    for i in range(len(matrix_documents)):
        # Calcula la similitud coseno
        similarity = calculate_cosine_similarity(vector_query, matrix_documents[i])
        # Agrega la similitud a la lista de similitudes
        similarities[i] = similarity
    return sort_dictionary_by_value(similarities)

def sort_dictionary_by_value(dictionary):
    # Ordena el diccionario
    sorted_dictionary = {k: v for k,v in sorted(dictionary.items(), key=lambda x: x[1], reverse=True)}
    return sorted_dictionary


In [4]:
# Vectorizar los documentos
vocab=create_vocabulary(cuentos)
vocab_tfidf=create_vocabulary_tfidf(cuentos) #nuevo vocabulario codificado
mat_vec_tfidf=create_matrix_documents_tfidf(cuentos, vocab_tfidf)

In [5]:
querys=['bruja mala','princesa dragon','rey maldición','pantano hermoso','vestido largo']
querys_p=['navidad brillante','animal hermoso'
          ,'cabellos dorados','pequeño ladrón','calabozo ogro']
q_t=querys+querys_p
for xx in range(len(q_t)):
    print('Query: ',q_t[xx])
    q_res=calculate_similarity_for_query(q_t[xx]
                                         ,mat_vec_tfidf,vocab) # es el mismo vocabulario y matriz de valores
    for cc in list(q_res.keys()):
        q_res[re.sub('.txt','',cuentos_lista[int(cc)])]=q_res[cc]
        del q_res[cc]
    print(q_res)
    print(' ')

Query:  bruja mala
{'Aurora, la princesa que no conocía la luna': 0.08996859661192652, 'El mejor regalo de Santa': 0.014772047372428439, 'La princesa más bella del reino': 0.011465598548998764, 'La princesa rizos largos': 0.008915418525956325, 'Pulgarcito': 0.008092547148513598, 'CaperucitaRoja': 0.00796710568382442, 'Platero y yo': 0.007647264823575657, 'El gato con botas': 0.00761249741881608, 'El Jorobado de Notre Dame': 0.007537142594362843, 'Ana y el Caballito Verde': 0.0, 'Como se hizo la lluvia': 0.0, 'El abad y los tres enigmas': 0.0, 'El agua mágica para el rey': 0.0, 'El carrete mágico': 0.0, 'El cerdito de color verde': 0.0, 'El deseo de Luisa': 0.0, 'El gato que soñaba con alcanzar la luna': 0.0, 'El gigante egoísta': 0.0, 'El Hada que no podía volar': 0.0, 'El leñador honrado': 0.0, 'El león que se creía cordero': 0.0, 'El libro de la selva': 0.0, 'El mago de Oz': 0.0, 'El pájaro flautista': 0.0, 'El patito feo': 0.0, 'El perrito Junior': 0.0, 'El perro cazador y su amo': 

# Método probabilistico 

Libro: pagina 37. *Natural Language processing for online applications. Text retrieval, extraction and categorization*
**Probabilistic ranking**

Se define a partir de la idea de teoria de probabilidad. Donde primeramente se debe definir la probabilidad de que un documento sea relevante ($R$) dado un *Query* $Q$ con $X \in {0,1}$.

$$P(R_Q=X|D) $$

Para posteriormente con la ley de Bayes obtener
$$ \frac{P(R_Q=1|D)}{P(R_Q=0|D)} $$

## Método Probabilistico Clásico

Entonces una vez hecho esto, se obtiene que:

$$IDF_t=\log(\frac{N-n_t+0.5}{n_t+0.5}) $$
donde:
* $N$ es el tamaño de la colección (corpus o conjunto de documentos)
* $n_t$ es el numero de documentos que contienen el termino de la query $t$



$$ TF=\frac{f(k+1)}{f+KL}$$
donde: 
* $L$ es la longitud normalizada del documento D.
* $K$ es una constante entre $1$ y $2$
* $f$ es la frecuencia de la palabra del Query

<div class="alert alert-block alert-info">
Cabe mencionar que con este método hay un detalle, el cual consiste en una excepción en la relación que existe entre las variables $n$ y $N$ donde en caso de que no se cumpla la relación, se debe asumir que la multiplicación es 0 o 1 como sea requerido.

$$ f(x)= \left\{ \begin{array}{lcc}
             TF\times IDF &   si  & 2n < N \\
             \\ 0 || 1 &  si & O.C. 
             \end{array}
   \right. $$

Todo esto con el fin de modular su efecto en darle más credito a los documentos más cortos
</div>
obteniendo un peso $\omega$ en el Query de la forma $Q={t_1,t_2,\cdots,t_m}$:

<div class="alert alert-block alert-success">
$$\omega_{t,d}=\frac{f(k+1)}{f+KL}\log(\frac{N-n_t+0.5}{n_t+0.5}) $$
</div>

## Forma convexa 

Se tiene la siguiente Formula para el modo convexo.

<div class="alert alert-block alert-success">
$$\omega_{t,d}= \alpha +(1-\alpha)\frac{f}{f+0.5+KL}(\frac{\log(\frac{N+0.5}{n})}{\log(N+1)})$$

donde:

* $\alpha$ es una constante que expresa que a pesar de que un termino $t$ no aparece en el documento, su probabilidad de ocurrencia no es cero, mientras que $(1-\alpha)$ pesa la contribución $TF\times IDF$
</div>
<div class="alert alert-block alert-info">
    <b>El término $L$:</b> 
    Se encontraron diferentes formulas para representar  
    
    
<span style='color:#A52A2A'>**La primer fórmula para calcular la longitud normalizada del documento que se encontró es:**</span>

\begin{equation}
L_1 = K \cdot (1 - b +  ( b \cdot \frac{\text{len(doc)}}{\text{avg_len}}))
\end{equation}

donde:

* $K$ es una constante que se usa para normalizar la longitud del documento. En este modelo, $K$ se define como $1 - b + b \cdot (\frac{\text{avg_len}}{\text{corpus_len}})$.
* $b$ es un factor de ajuste.
* $\text{len(doc)}$ es la longitud del documento actual que se está procesando.
* $\text{avg_len}$ es la longitud promedio del documento en el corpus **(promedio de todas las palabras en los documentos)**.
* $\text{corpus_len}$ es el número total de términos en el corpus **(total de palabras que tienen todos los documentos)**.


Esta fórmula se utiliza en el modelo de espacio vectorial para la recuperación de información y ayuda a mejorar la precisión de la búsqueda mediante la normalización de la longitud del documento.

<span style='color:#A52A2A'>**La segunda fórmula para calcular la longitud normalizada del documento que se encontró es:**</span>
\begin{equation}
L_2 = \frac{1}{\sqrt{\text{len(words)}}} \cdot \sum_{i=1}^{\text{len(words)}} \text{tf}_i
\end{equation}

donde:


* $L$ es la longitud normalizada del documento.
* $\text{len(words)}$ es la cantidad de palabras en el documento.
* $\text{tf}$ es la frecuencia de términos para cada palabra en el documento.


La raíz cuadrada de $\text{len(words)}$ en la fórmula es para normalizar la longitud del documento en función de su tamaño. La suma de $\text{tf}$ es para contar la frecuencia total de términos en el documento.    
    
</div>

In [6]:
# Obtener vocabulario
def create_vocabulary(list_of_documents):
    i = 0
    vocabulary = {}
    # Para cada documento en la lista de documentos
    for document in list_of_documents:
        # Separa el documento en palabras
        words = document.split(' ')
        #words=document
        # Para cada palabra en las palabras
        for word in words:
            # Si la palabra no está en el vocabulario
            if word not in vocabulary:
                vocabulary[word] = i
                i += 1
    return vocabulary

def n_t(documents,term):
    nt=0
    for ab in range(len(term)):
        for a in range(len(documents)):
            if set(term[ab].split()).intersection(documents[a].split()):
                nt=nt+1
    return nt

def L1(K,b,doc,documents):
    len_doc=len(doc)
    avg_len=0
    temp=0
    for a in range(len(documents)):
        temp=temp+len(documents[a].split())
    avg_len=temp/len(documents)
    return K*((1-b)+(b*(len_doc/avg_len)))

def L2(doc,query):
    tf2=0
    
    for a in query:
        for b in doc.split():
            if a==b:
                tf2+=1
    return (1/(np.sqrt(len(doc))))*tf2
    
def create_vectorized_document(document, vocabulary,alpha,K,L,N,n_t):
    # Separa el documento en palabras
    words = document.split(' ')
    vector2=[]
    # Crea un vector de ceros
    vector = np.zeros(len(vocabulary))
    ren=vector.shape
    # Para cada palabra en las palabras
    for word in words:
        # Si la palabra está en el vocabulario
        if word in vocabulary:
            # Incrementa el valor del vector en la posición de la palabra
            vector[vocabulary[word]] += 1
    for aa in range(ren[0]): #Método de INQUERY
        tff=(1-alfa)*(vector[aa,]/(vector[aa,]+0.5+(K*L)))
        idff=((np.log((N+0.5)/n_t))/(np.log(N+1)))
        vector2.append(alpha+(tff*idff))
        
    return vector2


def create_matrix_documents(list_of_documents, vocabulary,alpha,K,L,N,query):
    # Crea una matriz de ceros
    matrix = np.zeros((len(list_of_documents), len(vocabulary)))
    # Para cada documento en la lista de documentos
    nt=n_t(list_of_documents,query)
    for i in range(len(list_of_documents)):
        # Crea un vector para el documento
        vector = create_vectorized_document(list_of_documents[i], vocabulary,alpha,K,L,N,nt)
        # Agrega el vector a la matriz
        matrix[i] = vector
    return matrix


## Comparación de las querys con similaridad de cosenos

def calculate_cosine_similarity(vector1, vector2):
    # Multiplica los vectores
    dot_product = np.dot(vector1, vector2)
    # Calcula la norma de los vectores
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    # Calcula la similitud coseno
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity
# Vector para query
def create_vectorized_documentq(document, vocabulary):
    # Separa el documento en palabras
    words = document.split(' ')
    # Crea un vector de ceros
    vector = np.zeros(len(vocabulary))
    # Para cada palabra en las palabras
    for word in words:
        # Si la palabra está en el vocabulario
        if word in vocabulary:
            # Incrementa el valor del vector en la posición de la palabra
            vector[vocabulary[word]] += 1
    return vector

# Calcula la similaridad de cosenos
def calculate_similarity_for_query(query, matrix_documents, vocabulary):
    # Crea un vector para la consulta
    vector_query = create_vectorized_documentq(query, vocabulary)
    # Crea una lista para las similitudes
    similarities = {}
    # Para cada documento en la matriz de documentos
    for i in range(len(matrix_documents)):
        # Calcula la similitud coseno
        similarity = calculate_cosine_similarity(vector_query, matrix_documents[i])
        # Agrega la similitud a la lista de similitudes
        similarities[i] = similarity
    return sort_dictionary_by_value(similarities)

def sort_dictionary_by_value(dictionary):
    # Ordena el diccionario
    sorted_dictionary = {k: v for k,v in sorted(dictionary.items(), key=lambda x: x[1], reverse=True)}
    return sorted_dictionary

In [7]:
querys=['bruja mala','princesa dragon','rey maldición','pantano hermoso','vestido largo']
querys_p=['navidad brillante','animal hermoso'
          ,'cabellos dorados','pequeño ladrón','calabozo ogro']
q_t=querys+querys_p

### Valores (hiperparametros)
alfa=0
beta=0.34
K=1
###
vocab=create_vocabulary(cuentos)
m_doc=create_matrix_documents(cuentos,vocab,alfa,K,L2(cuentos[0],q_t),len(cuentos),q_t)

In [8]:

for xx in range(len(q_t)):
    print('Query: ',q_t[xx])
    q_res2=calculate_similarity_for_query(q_t[xx],m_doc,vocab) # es el mismo vocabulario y matriz de valores
    for cc in list(q_res2.keys()):
        q_res2[re.sub('.txt','',cuentos_lista[int(cc)])]=q_res2[cc]
        del q_res2[cc]
    print(q_res2)
    print(' ')

Query:  bruja mala
{'Ana y el Caballito Verde': 0.0, 'Como se hizo la lluvia': 0.0, 'El abad y los tres enigmas': 0.0, 'El agua mágica para el rey': 0.0, 'El carrete mágico': 0.0, 'El cerdito de color verde': 0.0, 'El deseo de Luisa': 0.0, 'El gato con botas': 0.0, 'El gato que soñaba con alcanzar la luna': 0.0, 'El gigante egoísta': 0.0, 'El Hada que no podía volar': 0.0, 'El Jorobado de Notre Dame': 0.0, 'El leñador honrado': 0.0, 'El león que se creía cordero': 0.0, 'El libro de la selva': 0.0, 'El mejor regalo de Santa': 0.0, 'El pájaro flautista': 0.0, 'El perrito Junior': 0.0, 'El Príncipe y la cebolla': 0.0, 'El ratón y el león': 0.0, 'El soldadito de plomo': 0.0, 'El traje nuevo del emperador': 0.0, 'Garbancito': 0.0, 'Juan sin miedo': 0.0, 'La gallina roja': 0.0, 'La gata encantada': 0.0, 'La gratitud de la fiera': 0.0, 'La nuez de Oro': 0.0, 'La princesa más bella del reino': 0.0, 'La princesa rizos largos': 0.0, 'La ratita blanca': 0.0, 'La ratita presumida': 0.0, 'Las 3 pri

# Conclusiones

Se observó que con el método probabilistico no se tienen valores tan altos como en el método ranked, también se observa que al cambiar el parámetro $\alpha$ se tiene una variación importante en los datos, por lo que su selección debe ser totalmente importante. 
Tambien se observa poca variabilidad en los valores probabilisticos, donde los rangos no superan 0.01%.
Aún asi, este método convexo tiene mejor desempeño por la sensibilización al tamaño de los textos y de la cantidad de documentos en los que aparace cada Query, por lo que se tiene esa ventaja de que la longitud del texto aportará de manera diferente a la probabilidad que se tenga. Se observó que no hubo diferencia notable en el orden de probabilidad al cambiar el hiperparámetro $\alpha$.